**Import required modules**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

**Setup TPU**

In [ ]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU:", resolver.cluster_spec().as_dict())
except ValueError:
    strategy = tf.distribute.get_strategy()
print(f"Running on {strategy.num_replicas_in_sync} replicas")

Running on TPU: {}
Running on 8 replicas


**Define Batch Size**

In [ ]:
batch_size = 256

**Paths for Annotations**

In [ ]:
train_annotation_path = "Annotation/Train_annotation.csv"
val_annotation_path = "Annotation/Val_annotation.csv"

**Load Annotations**

In [ ]:
train_annotations = pd.read_csv(train_annotation_path)
val_annotations = pd.read_csv(val_annotation_path)

<ipython-input-6-249c258d7047>:2: DtypeWarning: Columns (33,38,43,48) have mixed types. Specify dtype option on import or set low_memory=False.
  train_annotations = pd.read_csv(train_annotation_path)
<ipython-input-6-249c258d7047>:3: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  val_annotations = pd.read_csv(val_annotation_path)


**Update file paths and append extension**

In [ ]:
train_annotations['File_location'] = "Train/" + train_annotations['File_location'] + ".JPEG"
val_annotations['File_location'] = "val/" + val_annotations['File_location'] + ".JPEG"


**Process Labels**

In [ ]:
train_labels_list = list(map(lambda x: x.title(), train_annotations['Name_1'].tolist()))
val_labels_list = list(map(lambda x: x.title(), val_annotations['Name_1'].tolist()))

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels_list)
val_labels_encoded = label_encoder.transform(val_labels_list)

**Function for preprocessing**

In [ ]:
# Image loading function
def load_and_preprocess_image(file_path, bbox):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Extract bounding box coordinates
    xmin, ymin, xmax, ymax = tf.unstack(bbox)

    # Convert bounding box to integer and crop
    xmin = tf.cast(xmin, tf.int32)
    ymin = tf.cast(ymin, tf.int32)
    width = tf.cast(xmax - xmin, tf.int32)
    height = tf.cast(ymax - ymin, tf.int32)

    image = tf.image.crop_to_bounding_box(image, ymin, xmin, height, width)

    # Resize the image to (299, 299) for Xception
    image = tf.image.resize(image, (299, 299))

    return image

**Create datasets for training and validation**

In [ ]:
AUTO = tf.data.AUTOTUNE

def create_dataset(image_paths, labels, bbox_annotations, is_training=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels, bbox_annotations))

    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.map(lambda x, y, bbox: (load_and_preprocess_image(x, bbox), y),
                          num_parallel_calls=AUTO)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.map(lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y),
                          num_parallel_calls=AUTO)
    return dataset.prefetch(AUTO)

# Bounding Box information
train_bboxes = train_annotations[['Xmin_1', 'Ymin_1', 'Xmax_1', 'Ymax_1']].values.tolist()
val_bboxes = val_annotations[['Xmin_1', 'Ymin_1', 'Xmax_1', 'Ymax_1']].values.tolist()

train_ds = create_dataset(train_annotations['File_location'].tolist(),
                          train_labels_encoded, train_bboxes, is_training=True)
validation_ds = create_dataset(val_annotations['File_location'].tolist(),
                               val_labels_encoded, val_bboxes)

# Check dataset sizes
print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of validation samples: {validation_ds.cardinality()}")

Number of training samples: 851
Number of validation samples: 77


**Create Model**

In [ ]:
# Create model within TPU strategy scope
with strategy.scope():
    inputs = tf.keras.Input(shape=(299, 299, 3))
    base_model = tf.keras.applications.Xception(
        weights="imagenet",
        input_shape=(299, 299, 3),
        include_top=False,
        pooling='max'
    )
    base_model.trainable = False

    x = base_model(inputs, training=False)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(len(label_encoder.classes_), activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

83683744/83683744 [==============================] - 0s 0us/step


**Save labels to text file**

In [ ]:
with tf.io.gfile.GFile("labels.txt", "w") as f:
    for l in label_encoder.classes_:
        f.write(f"{l}\n")

**Model Summary**

In [ ]:
model.summary(show_trainable=True)

Model: "model"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         Y          
                                                                            
 xception (Functional)       (None, 2048)              2086148   N          
                                                       0                    
                                                                            
 batch_normalization_4 (Bat  (None, 2048)              8192      Y          
 chNormalization)                                                           
                                                                            
 dense (Dense)               (None, 512)               1049088   Y          
                                                                            
 dropout (Dropout)           (None, 512)               0     

**Training**

In [ ]:
epochs = 50
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, mode='min'
        )
    ]
)

Epoch 1/50
851/851 [==============================] - 137s 141ms/step - loss: 7.1431 - accuracy: 0.0057 - val_loss: 5.5521 - val_accuracy: 0.5443 - lr: 1.0000e-04
Epoch 2/50
851/851 [==============================] - 110s 129ms/step - loss: 6.1379 - accuracy: 0.0948 - val_loss: 4.2288 - val_accuracy: 0.6920 - lr: 1.0000e-04
Epoch 3/50
851/851 [==============================] - 109s 127ms/step - loss: 5.3369 - accuracy: 0.2172 - val_loss: 3.1939 - val_accuracy: 0.7354 - lr: 1.0000e-04
Epoch 4/50
851/851 [==============================] - 109s 128ms/step - loss: 4.6144 - accuracy: 0.3332 - val_loss: 2.4796 - val_accuracy: 0.7507 - lr: 1.0000e-04
Epoch 5/50
851/851 [==============================] - 109s 127ms/step - loss: 3.9803 - accuracy: 0.4321 - val_loss: 2.0458 - val_accuracy: 0.7448 - lr: 1.0000e-04
Epoch 6/50
851/851 [==============================] - 108s 127ms/step - loss: 3.4563 - accuracy: 0.5093 - val_loss: 1.7855 - val_accuracy: 0.7375 - lr: 1.0000e-04
Epoch 7/50
851/851 [==

**Save the model**

In [ ]:
model.save("model.keras")